# 10. hét / Prológus

A mai órán a következőkről lesz szó:
- Idősorok elemzése 1D-CNN és RNN segítségével
- Zajcsökkentés konvolúciós rétegekből felépített Autoencoder segítségével
- Képszegmentáció haladó konvolúciós hálóval

# 10. hét / I. Idősorok elemzése

### Szükséges Importok

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, Conv1D, MaxPooling1D, LSTM
from tensorflow.keras.optimizers import SGD
import matplotlib.pyplot as plt

### 1. Adatgyűjtés

In [ ]:
# Adatok elkészítése


plt.figure(figsize=(12,8))
plt.plot(X)

### 2-3. Adatok feltérképezése és preprocesszálása

Itt nem lényeges.

### 4. Modell választása

In [ ]:
# 1D konvolúció alapú háló
def make_1d_convnet():

# LSTM alapú háló
def make_LSTM():


In [ ]:
def make_timeseries_instances(timeseries, window_size):
    timeseries = np.asarray(timeseries)

    assert 0 < window_size < timeseries.shape[0] , "Out of range 0 < {} < {} ".format(window_size,timeseries.shape[0])
    
    X = np.atleast_3d(np.array([timeseries[start:start + window_size] for start in range(0, timeseries.shape[0] - window_size)]))
    Y = timeseries[window_size:]
    return X, Y

In [ ]:
def evaluate_timeseries(timeseries, window_size, valid_split=0.15, test_split=0.15):
    filter_length = 5
    nb_filter = 4

    timeseries = np.atleast_2d(timeseries)
    if timeseries.shape[0] == 1:
        timeseries = timeseries.T
    nb_samples, nb_series = timeseries.shape
    model = make_1d_convnet(window_size=window_size, filter_length=filter_length, nb_input_series=nb_series, nb_outputs=nb_series, nb_filter=nb_filter)
    model.summary()

    # Preprocessing és dataset splitting
    X, Y = make_timeseries_instances(timeseries, window_size)
    
    valid_size = int(nb_samples*(1-test_split-valid_split))
    test_size = int(nb_samples*(1-test_split))
    X_train, Y_train = X[:valid_size], Y[:valid_size]
    X_valid, Y_valid = X[valid_size:test_size], Y[valid_size:test_size]
    X_test, Y_test   = X[test_size:], Y[test_size:]

    # Modell illesztése
    model.fit(X_train, Y_train, epochs=50, batch_size=16, validation_data=(X_valid, Y_valid), verbose=2)
    
    # Modell kiértékelése
    preds = model.predict(X_test)

    return Y_test, preds  

### 5. Modell illesztése

In [ ]:
window_size = 20
targets, preds = evaluate_timeseries(X, window_size)

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(preds, color='r', label="Predictions")
plt.plot(targets, color='g', label="Targets")
plt.legend()

# 10. hét / II. Autoencoder alapú zajcsökkentés

### Szükséges Importok

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras

from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

### 1. Adatgyűjtés

In [ ]:
# Tanítóadatok
(x_train, _) ,(x_test, _) = 

print(x_train.shape, x_test.shape)

# Dataset splitting

# Data normalizing

print(x_train.shape, x_valid.shape, x_test.shape)

### 2-3. Adatfeltérképezés és data preprocessing

In [ ]:
# Zaj hozzáadása

# Klippelés 0 és 1 közé

# Vizualizáció
n = 8
plt.figure(figsize=(16, 4))
for i in range(n):
  # eredeti
  ax = plt.subplot(1, n, i + 1)
  ax.set_title('zajos')
  plt.imshow(x_test_noisy[i].reshape(28,28))
  plt.gray()

### 4. Modell választás

In [ ]:
autoencoder = 

autoencoder.summary()

autoencoder.compile(optimizer = 'adam', loss = 'mean_squared_error')

### 5. Modell illesztés

In [ ]:
autoencoder.fit(x_train_noisy, x_train,
               epochs = 10,
               batch_size = 128,
               shuffle = True,
               validation_data = (x_valid_noisy, x_valid))

In [ ]:
decoded_imgs = autoencoder.predict(x_test_noisy)

### 6. Kiértékelés

In [ ]:
n = 8
plt.figure(figsize=(16, 4))
for i in range(n):
  # eredeti
  ax = plt.subplot(3, n, i + 1)
  ax.set_title('eredeti')
  plt.imshow(x_test[i].reshape(28,28))
  plt.gray()

  # zajos
  ax = plt.subplot(3, n, i + 1 + n)
  ax.set_title('eredeti')
  plt.imshow(x_test_noisy[i].reshape(28,28))
  plt.gray()

  # visszaállított
  ax = plt.subplot(3, n, i + 1 + 2 * n)
  ax.set_title('visszaállított')
  plt.imshow(decoded_imgs[i].reshape(28,28))
  plt.gray()
plt.show()

### 7. Modell finomítása

In [ ]:
noise_factor = 0.4
x_test_noisy = x_test + noise_factor * np.random.lognormal(0, 1, size=x_test.shape)
x_test_noisy = np.clip(x_test_noisy, 0, 1)

decoded_imgs = autoencoder.predict(x_test_noisy)

# eredeti, zajos és visszaállított képek kirajzolása
n = 8
plt.figure(figsize=(16, 4))
for i in range(n):
  # eredeti
  ax = plt.subplot(3, n, i + 1)
  ax.set_title('eredeti')
  plt.imshow(x_test[i].reshape(28,28))
  plt.gray()

  # zajos
  ax = plt.subplot(3, n, i + 1 + n)
  ax.set_title('eredeti')
  plt.imshow(x_test_noisy[i].reshape(28,28))
  plt.gray()

  # visszaállított
  ax = plt.subplot(3, n, i + 1 + 2 * n)
  ax.set_title('visszaállított')
  plt.imshow(decoded_imgs[i].reshape(28,28))
  plt.gray()
plt.show()

# 10. hét / III. Képszegmentáció haladó konvolúciós háló segítségével

### Szükséges könyvtárak
1. **TensorFlow: Datasets** - Olyan könyvtár, amelyben több, előre összeállított és feldolgozott adatbázis található
2. **TensorFlow: Examples** - TensorFlow oktatóanyagok és adathalmazok tárolására létrehozott könyvtár
3. **Pydot** - Neurális hálók vizaulizációjához szükséges könyvtár

In [ ]:
%pip install tensorflow-datasets
%pip install tensorflow-examples
%pip install pydot

In [ ]:
#%pip install -q git+https://github.com/tensorflow/examples.git

### Szükséges importok

In [ ]:
# Deep Learning keretrendszer
import tensorflow as tf
import tensorflow_datasets as tfds

# Adathalmaz
from tensorflow_examples.models.pix2pix import pix2pix

# Megjelenítés
from IPython.display import clear_output
import matplotlib.pyplot as plt

### 1. Adatgyűjtés

### 2. Adatfeltérképezés + 3. Adat előkészítés

In [ ]:
# Vizuális adatot noramlizáljuk 0 és 1 közé
def normalize(input_image, input_mask):

# Az adatok betöltése
def load_image(datapoint):
  # Érdemes átskálázni a bemeneti kép méretét

  # Azért, hogy ne 'híguljon' fel az adat, nearest_neighbor algoritmust használunk

  # A betöltés végén a képet normalizáljuk

  # Visszatérünk a preprocesszált képekkel

In [ ]:
# Betöltjük az adatot


In [ ]:
# Augmentáció
class Augment(tf.keras.layers.Layer):
  def __init__(self, seed=42):
    super().__init__()
    # Ha ugyanazzal a random seeddel dolgozunk, akkor ugyanazok fognak megfordulni
    
  def call(self, inputs, labels):

In [ ]:
# Előkészítés a tanításhoz
TRAIN_LENGTH = info.splits['train'].num_examples
BATCH_SIZE = 64
BUFFER_SIZE = 1000
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

train_batches = (
    train_images
    .cache()
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
    .repeat()
    .map(Augment())
    .prefetch(buffer_size=tf.data.AUTOTUNE))

test_batches = test_images.batch(BATCH_SIZE)

In [ ]:
# Megjelenítés
def display(display_list):
  plt.figure(figsize=(15, 15))

  title = ['Input Image', 'True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    plt.imshow(tf.keras.utils.array_to_img(display_list[i]))
    plt.axis('off')
  plt.show()

for images, masks in train_batches.take(1):
  sample_image, sample_mask = images[0], masks[0]
  display([sample_image, sample_mask])

### 4. Modellválasztás

!["unet.jpg"](unet.jpg)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=[128, 128, 3], include_top=False)

# "Legózás"
layer_names = [
    'block_1_expand_relu',   # 64x64
    'block_3_expand_relu',   # 32x32
    'block_6_expand_relu',   # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',      # 4x4
]
base_model_outputs = [base_model.get_layer(name).output for name in layer_names]

# Contracting (Feature Extractor) path létrehozása
down_stack = tf.keras.Model(inputs=base_model.input, outputs=base_model_outputs)

down_stack.trainable = False

# Expansive path létrehozása
up_stack = [
    pix2pix.upsample(512, 3),  # 4x4 -> 8x8
    pix2pix.upsample(256, 3),  # 8x8 -> 16x16
    pix2pix.upsample(128, 3),  # 16x16 -> 32x32
    pix2pix.upsample(64, 3),   # 32x32 -> 64x64
]

In [ ]:
def unet_model(output_channels:int):
  inputs = tf.keras.layers.Input(shape=[128, 128, 3])

  # Downsampling
  skips = down_stack(inputs)
  x = skips[-1]
  skips = reversed(skips[:-1])

  # Upsampling
  for up, skip in zip(up_stack, skips):
    x = up(x)
    concat = tf.keras.layers.Concatenate()
    x = concat([x, skip])

  # Utolsó réteg
  last = tf.keras.layers.Conv2DTranspose(
      filters=output_channels, kernel_size=3, strides=2,
      padding='same')  #64x64 -> 128x128

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
OUTPUT_CLASSES = 3

model = unet_model(output_channels=OUTPUT_CLASSES)
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
def create_mask(pred_mask):
  pred_mask = tf.math.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

def show_predictions(dataset=None, num=1):
  if dataset:
    for image, mask in dataset.take(num):
      pred_mask = model.predict(image)
      display([image[0], mask[0], create_mask(pred_mask)])
  else:
    display([sample_image, sample_mask,
             create_mask(model.predict(sample_image[tf.newaxis, ...]))])

In [ ]:
show_predictions()

In [ ]:
class DisplayCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    clear_output(wait=True)
    show_predictions()
    print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

### 5. Modell illesztése

In [ ]:
EPOCHS = 40
VAL_SUBSPLITS = 5
VALIDATION_STEPS = info.splits['test'].num_examples//BATCH_SIZE//VAL_SUBSPLITS

model_history = model.fit(train_batches,
                           epochs=EPOCHS,
                          steps_per_epoch=STEPS_PER_EPOCH,
                          validation_steps=VALIDATION_STEPS,
                          validation_data=test_batches,
                          callbacks=[DisplayCallback()])

### 6. Modell kiértékelése

In [ ]:
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

plt.figure()
plt.plot(model_history.epoch, loss, 'r', label='Training loss')
plt.plot(model_history.epoch, val_loss, 'bo', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.ylim([0, 1])
plt.legend()
plt.show()

In [ ]:
show_predictions(test_batches, 3)